XML 관계 데이터 생성 함수 정의

In [1]:
# ------------------------------------------------------
# XML 기반 직원 관계 데이터 생성 시스템
# ------------------------------------------------------

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from xml.dom import minidom
from pathlib import Path
import json
from datetime import datetime

# 설정
DATA_PATH = Path("./data/IBM_HR_personas_assigned.csv")
XML_OUTPUT_PATH = Path("./data/employee_relationships.xml")
JSON_OUTPUT_PATH = Path("./data/employee_relationships.json")  # JSON 버전도 생성
RNG_SEED = 42
np.random.seed(RNG_SEED)

def prettify_xml(elem):
    """XML을 보기 좋게 포매팅"""
    rough_string = ET.tostring(elem, 'unicode')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

print("XML 기반 관계 데이터 생성 시스템 초기화 완료")

XML 기반 관계 데이터 생성 시스템 초기화 완료


관계 생성 로직 함수들

In [2]:
# ------------------------------------------------------
# 각 관계 유형별 생성 로직
# ------------------------------------------------------

def generate_hierarchy_relationships(df):
    """계층 관계 생성 (상사-부하)"""
    hierarchy_rels = {}
    
    for dept in df['Department'].unique():
        dept_employees = df[df['Department'] == dept].copy()
        dept_employees = dept_employees.sort_values(['JobLevel', 'YearsAtCompany'], 
                                                  ascending=[False, False])
        
        # 직급별 관리자-부하 매핑
        for high_level in [4, 5]:
            managers = dept_employees[dept_employees['JobLevel'] == high_level]
            subordinates = dept_employees[dept_employees['JobLevel'] == high_level-1]
            
            if len(managers) == 0 or len(subordinates) == 0:
                continue
                
            for _, manager in managers.iterrows():
                # 관리 범위 결정
                span_of_control = min(np.random.randint(3, 8), len(subordinates))
                selected_subs = subordinates.sample(
                    n=min(span_of_control, len(subordinates)),
                    random_state=RNG_SEED + manager['EmployeeNumber']
                )
                
                mgr_id = str(manager['EmployeeNumber'])
                if mgr_id not in hierarchy_rels:
                    hierarchy_rels[mgr_id] = {'manages': [], 'reports_to': None}
                
                for _, sub in selected_subs.iterrows():
                    sub_id = str(sub['EmployeeNumber'])
                    
                    # 관리자 측면: 부하직원 리스트
                    hierarchy_rels[mgr_id]['manages'].append({
                        'employee_id': sub_id,
                        'employee_name': f"Employee_{sub_id}",
                        'job_role': sub['JobRole'],
                        'job_level': int(sub['JobLevel']),
                        'relationship_start': '2023-01-01',
                        'management_style': np.random.choice(['collaborative', 'directive', 'supportive']),
                        'span_of_control': span_of_control
                    })
                    
                    # 부하직원 측면: 상사 정보
                    if sub_id not in hierarchy_rels:
                        hierarchy_rels[sub_id] = {'manages': [], 'reports_to': None}
                    
                    hierarchy_rels[sub_id]['reports_to'] = {
                        'manager_id': mgr_id,
                        'manager_name': f"Employee_{mgr_id}",
                        'manager_role': manager['JobRole'],
                        'manager_level': int(manager['JobLevel']),
                        'reporting_since': '2023-01-01',
                        'reporting_frequency': np.random.choice(['daily', 'weekly', 'bi-weekly'])
                    }
    
    return hierarchy_rels

def generate_collaboration_relationships(df):
    """협업 관계 생성"""
    collab_rels = {}
    
    for dept in df['Department'].unique():
        dept_employees = df[df['Department'] == dept]
        
        for i, emp1 in dept_employees.iterrows():
            emp1_id = str(emp1['EmployeeNumber'])
            if emp1_id not in collab_rels:
                collab_rels[emp1_id] = []
            
            for j, emp2 in dept_employees.iterrows():
                if i >= j:
                    continue
                
                emp2_id = str(emp2['EmployeeNumber'])
                
                # 협업 강도 계산
                level_diff = abs(emp1['JobLevel'] - emp2['JobLevel'])
                base_strength = max(0.2, 1.0 - level_diff * 0.15)
                
                if emp1['JobRole'] == emp2['JobRole']:
                    base_strength *= 1.3
                
                final_strength = max(0.1, min(1.0, base_strength * np.random.uniform(0.7, 1.3)))
                
                # 임계값 이상인 관계만 저장
                if final_strength >= 0.4:
                    collaboration_info = {
                        'colleague_id': emp2_id,
                        'colleague_name': f"Employee_{emp2_id}",
                        'colleague_role': emp2['JobRole'],
                        'colleague_level': int(emp2['JobLevel']),
                        'collaboration_strength': round(final_strength, 3),
                        'interaction_frequency': round(np.random.beta(2, 5), 3),
                        'collaboration_type': 'same_department',
                        'common_projects': np.random.randint(1, 4),
                        'relationship_quality': np.random.choice(['excellent', 'good', 'fair'])
                    }
                    
                    collab_rels[emp1_id].append(collaboration_info)
                    
                    # 양방향 관계 추가
                    if emp2_id not in collab_rels:
                        collab_rels[emp2_id] = []
                    
                    collab_rels[emp2_id].append({
                        'colleague_id': emp1_id,
                        'colleague_name': f"Employee_{emp1_id}",
                        'colleague_role': emp1['JobRole'],
                        'colleague_level': int(emp1['JobLevel']),
                        'collaboration_strength': round(final_strength, 3),
                        'interaction_frequency': round(np.random.beta(2, 5), 3),
                        'collaboration_type': 'same_department',
                        'common_projects': np.random.randint(1, 4),
                        'relationship_quality': np.random.choice(['excellent', 'good', 'fair'])
                    })
    
    return collab_rels

def generate_project_relationships(df, num_projects=25):
    """프로젝트 팀 관계 생성"""
    project_rels = {}
    project_registry = []
    
    for project_id in range(1, num_projects + 1):
        project_code = f'PRJ_{project_id:03d}'
        
        # 프로젝트 타입 결정
        is_cross_dept = np.random.random() < 0.4
        team_size = np.random.randint(3, 9)
        
        if is_cross_dept:
            selected_depts = np.random.choice(df['Department'].unique(), size=min(2, len(df['Department'].unique())), replace=False)
            team_members = []
            for dept in selected_depts:
                dept_employees = df[df['Department'] == dept]
                dept_team_size = max(1, team_size // len(selected_depts))
                if len(dept_employees) > 0:
                    selected = dept_employees.sample(n=min(dept_team_size, len(dept_employees)), random_state=RNG_SEED + project_id)
                    team_members.append(selected)
            
            if team_members:
                team = pd.concat(team_members)
                project_type = 'cross_department'
            else:
                continue
        else:
            dept = np.random.choice(df['Department'].unique())
            dept_employees = df[df['Department'] == dept]
            team = dept_employees.sample(n=min(team_size, len(dept_employees)), random_state=RNG_SEED + project_id)
            project_type = 'single_department'
        
        # 프로젝트 정보 등록
        project_info = {
            'project_id': project_code,
            'project_name': f'Project_{project_code}',
            'project_type': project_type,
            'team_size': len(team),
            'start_date': '2023-01-15',
            'end_date': '2023-12-15',
            'status': 'active' if np.random.random() < 0.8 else 'completed',
            'priority': np.random.choice(['high', 'medium', 'low'])
        }
        project_registry.append(project_info)
        
        # 팀 멤버 간 관계 생성
        team_list = list(team['EmployeeNumber'].astype(str))
        
        for emp_id in team_list:
            if emp_id not in project_rels:
                project_rels[emp_id] = []
            
            # 같은 프로젝트의 다른 팀원들과의 관계
            teammates = [tid for tid in team_list if tid != emp_id]
            
            project_rels[emp_id].append({
                'project_id': project_code,
                'project_name': project_info['project_name'],
                'project_type': project_type,
                'role_in_project': np.random.choice(['lead', 'contributor', 'specialist', 'support']),
                'teammates': [
                    {
                        'teammate_id': tid,
                        'teammate_name': f'Employee_{tid}',
                        'collaboration_intensity': round(np.random.uniform(0.6, 1.0), 3)
                    }
                    for tid in teammates[:3]  # 최대 3명의 주요 팀동료만 표시
                ],
                'project_start': project_info['start_date'],
                'project_end': project_info['end_date'],
                'project_status': project_info['status'],
                'contribution_level': round(np.random.uniform(0.5, 1.0), 3)
            })
    
    return project_rels, project_registry

print("관계 생성 함수들 정의 완료")

관계 생성 함수들 정의 완료


관계 데이터 생성 실행

In [3]:
# ------------------------------------------------------
# 모든 관계 데이터 생성 실행
# ------------------------------------------------------

# 데이터 로딩
df = pd.read_csv(DATA_PATH)
print(f"데이터 로딩 완료: {len(df)}명의 직원")

# 각 관계 유형별 데이터 생성
print("\n관계 데이터 생성 중...")

# 1. 계층 관계
hierarchy_relationships = generate_hierarchy_relationships(df)
print(f"계층 관계 생성 완료: {len(hierarchy_relationships)}명의 관계 정보")

# 2. 협업 관계  
collaboration_relationships = generate_collaboration_relationships(df)
print(f"협업 관계 생성 완료: {len(collaboration_relationships)}명의 관계 정보")

# 3. 프로젝트 관계
project_relationships, project_registry = generate_project_relationships(df, num_projects=25)
print(f"프로젝트 관계 생성 완료: {len(project_relationships)}명의 관계 정보")
print(f"프로젝트 등록: {len(project_registry)}개 프로젝트")

# 관계 통계
total_hierarchy_relations = sum(len(rels['manages']) for rels in hierarchy_relationships.values())
total_collaboration_relations = sum(len(rels) for rels in collaboration_relationships.values())
total_project_relations = sum(len(rels) for rels in project_relationships.values())

print(f"\n관계 통계:")
print(f"- 계층 관계: {total_hierarchy_relations}개")
print(f"- 협업 관계: {total_collaboration_relations}개")
print(f"- 프로젝트 관계: {total_project_relations}개")

데이터 로딩 완료: 1470명의 직원

관계 데이터 생성 중...
계층 관계 생성 완료: 371명의 관계 정보
협업 관계 생성 완료: 1470명의 관계 정보
프로젝트 관계 생성 완료: 122명의 관계 정보
프로젝트 등록: 25개 프로젝트

관계 통계:
- 계층 관계: 891개
- 협업 관계: 1098084개
- 프로젝트 관계: 138개


XML 구조 생성

In [4]:
# ------------------------------------------------------
# XML 구조 생성 및 데이터 입력
# ------------------------------------------------------

def create_employee_relationships_xml(df, hierarchy_rels, collab_rels, project_rels, project_registry):
    """직원별 관계 정보를 XML로 생성"""
    
    # 루트 엘리먼트 생성
    root = ET.Element("OrganizationNetwork")
    root.set("generated_date", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    root.set("total_employees", str(len(df)))
    root.set("data_source", "IBM_HR_personas_assigned.csv")
    
    # 프로젝트 레지스트리 섹션
    projects_section = ET.SubElement(root, "ProjectRegistry")
    for project in project_registry:
        project_elem = ET.SubElement(projects_section, "Project")
        for key, value in project.items():
            project_elem.set(key, str(value))
    
    # 직원 관계 섹션
    employees_section = ET.SubElement(root, "Employees")
    
    for _, emp in df.iterrows():
        emp_id = str(emp['EmployeeNumber'])
        
        # 직원 기본 정보
        employee_elem = ET.SubElement(employees_section, "Employee")
        employee_elem.set("id", emp_id)
        employee_elem.set("name", f"Employee_{emp_id}")
        employee_elem.set("department", str(emp['Department']))
        employee_elem.set("job_role", str(emp['JobRole']))
        employee_elem.set("job_level", str(emp['JobLevel']))
        employee_elem.set("years_at_company", str(emp['YearsAtCompany']))
        employee_elem.set("risk_tier", str(emp['argmax_RiskTier']))
        employee_elem.set("persona", str(emp['argmax_Persona']))
        
        # 계층 관계 (Hierarchy Relationships)
        hierarchy_elem = ET.SubElement(employee_elem, "HierarchyRelationships")
        
        if emp_id in hierarchy_rels:
            hierarchy_info = hierarchy_rels[emp_id]
            
            # 상사 정보
            if hierarchy_info['reports_to']:
                manager_elem = ET.SubElement(hierarchy_elem, "ReportsTo")
                for key, value in hierarchy_info['reports_to'].items():
                    manager_elem.set(key, str(value))
            
            # 부하직원 정보
            if hierarchy_info['manages']:
                subordinates_elem = ET.SubElement(hierarchy_elem, "Manages")
                for subordinate in hierarchy_info['manages']:
                    sub_elem = ET.SubElement(subordinates_elem, "Subordinate")
                    for key, value in subordinate.items():
                        sub_elem.set(key, str(value))
        
        # 협업 관계 (Collaboration Relationships)
        collab_elem = ET.SubElement(employee_elem, "CollaborationRelationships")
        
        if emp_id in collab_rels:
            for colleague in collab_rels[emp_id]:
                colleague_elem = ET.SubElement(collab_elem, "Colleague")
                for key, value in colleague.items():
                    colleague_elem.set(key, str(value))
        
        # 프로젝트 관계 (Project Relationships)
        project_elem = ET.SubElement(employee_elem, "ProjectRelationships")
        
        if emp_id in project_rels:
            for project in project_rels[emp_id]:
                proj_elem = ET.SubElement(project_elem, "ProjectParticipation")
                
                # 프로젝트 기본 정보
                for key, value in project.items():
                    if key != 'teammates':
                        proj_elem.set(key, str(value))
                
                # 팀동료 정보
                if 'teammates' in project:
                    teammates_elem = ET.SubElement(proj_elem, "Teammates")
                    for teammate in project['teammates']:
                        teammate_elem = ET.SubElement(teammates_elem, "Teammate")
                        for key, value in teammate.items():
                            teammate_elem.set(key, str(value))
    
    return root

# XML 생성 실행
print("XML 구조 생성 중...")
xml_root = create_employee_relationships_xml(
    df, hierarchy_relationships, collaboration_relationships, 
    project_relationships, project_registry
)

print("XML 구조 생성 완료")

XML 구조 생성 중...
XML 구조 생성 완료


XML 및 JSON 파일 저장

In [5]:
# ------------------------------------------------------
# XML 및 JSON 파일 저장
# ------------------------------------------------------

def save_xml_file(root_element, file_path):
    """XML 파일을 보기 좋게 포매팅해서 저장"""
    # 예쁜 포매팅으로 저장
    xml_str = prettify_xml(root_element)
    
    # UTF-8 인코딩으로 저장
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(xml_str)
    
    print(f"XML 파일 저장 완료: {file_path}")

def create_json_summary(df, hierarchy_rels, collab_rels, project_rels, project_registry):
    """JSON 형태의 요약 데이터도 생성"""
    json_data = {
        "metadata": {
            "generated_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_employees": len(df),
            "total_projects": len(project_registry),
            "data_source": "IBM_HR_personas_assigned.csv"
        },
        "projects": project_registry,
        "employee_relationships": {}
    }
    
    for _, emp in df.iterrows():
        emp_id = str(emp['EmployeeNumber'])
        
        employee_data = {
            "basic_info": {
                "name": f"Employee_{emp_id}",
                "department": emp['Department'],
                "job_role": emp['JobRole'],
                "job_level": int(emp['JobLevel']),
                "years_at_company": int(emp['YearsAtCompany']),
                "risk_tier": emp['argmax_RiskTier'],
                "persona": emp['argmax_Persona']
            },
            "relationships": {
                "hierarchy": hierarchy_rels.get(emp_id, {"manages": [], "reports_to": None}),
                "collaboration": collab_rels.get(emp_id, []),
                "projects": project_rels.get(emp_id, [])
            }
        }
        
        json_data["employee_relationships"][emp_id] = employee_data
    
    return json_data

# XML 파일 저장
save_xml_file(xml_root, XML_OUTPUT_PATH)

# JSON 파일도 생성 및 저장
json_data = create_json_summary(
    df, hierarchy_relationships, collaboration_relationships,
    project_relationships, project_registry
)

with open(JSON_OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print(f"JSON 파일 저장 완료: {JSON_OUTPUT_PATH}")

# 파일 크기 확인
xml_size = XML_OUTPUT_PATH.stat().st_size / (1024 * 1024)  # MB
json_size = JSON_OUTPUT_PATH.stat().st_size / (1024 * 1024)  # MB

print(f"\n파일 크기:")
print(f"- XML 파일: {xml_size:.2f} MB")
print(f"- JSON 파일: {json_size:.2f} MB")

XML 파일 저장 완료: data\employee_relationships.xml
JSON 파일 저장 완료: data\employee_relationships.json

파일 크기:
- XML 파일: 289.44 MB
- JSON 파일: 445.85 MB


XML 데이터 검증 및 미리보기

In [6]:
# ------------------------------------------------------
# 생성된 XML 데이터 검증 및 미리보기
# ------------------------------------------------------

def validate_and_preview_xml(xml_file_path, num_employees_to_show=3):
    """생성된 XML 파일 검증 및 미리보기"""
    
    try:
        # XML 파일 파싱
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        print(f"=== XML 파일 검증 결과 ===")
        print(f"루트 엘리먼트: {root.tag}")
        print(f"생성 일시: {root.get('generated_date')}")
        print(f"총 직원 수: {root.get('total_employees')}")
        
        # 프로젝트 레지스트리 확인
        projects_section = root.find('ProjectRegistry')
        if projects_section is not None:
            project_count = len(projects_section.findall('Project'))
            print(f"등록된 프로젝트: {project_count}개")
        
        # 직원 섹션 확인
        employees_section = root.find('Employees')
        if employees_section is not None:
            employees = employees_section.findall('Employee')
            print(f"XML에 포함된 직원: {len(employees)}명")
            
            # 샘플 직원들의 관계 정보 미리보기
            print(f"\n=== 샘플 직원 관계 정보 (상위 {num_employees_to_show}명) ===")
            
            for i, emp_elem in enumerate(employees[:num_employees_to_show]):
                emp_id = emp_elem.get('id')
                emp_name = emp_elem.get('name')
                dept = emp_elem.get('department')
                role = emp_elem.get('job_role')
                risk_tier = emp_elem.get('risk_tier')
                
                print(f"\n[직원 {i+1}: {emp_name}]")
                print(f"  부서: {dept}, 직무: {role}, 위험도: {risk_tier}")
                
                # 계층 관계
                hierarchy = emp_elem.find('HierarchyRelationships')
                if hierarchy is not None:
                    reports_to = hierarchy.find('ReportsTo')
                    manages = hierarchy.find('Manages')
                    
                    if reports_to is not None:
                        mgr_id = reports_to.get('manager_id')
                        mgr_role = reports_to.get('manager_role')
                        print(f"  상사: {mgr_id} ({mgr_role})")
                    
                    if manages is not None:
                        subordinates = manages.findall('Subordinate')
                        print(f"  부하직원: {len(subordinates)}명")
                
                # 협업 관계
                collab = emp_elem.find('CollaborationRelationships')
                if collab is not None:
                    colleagues = collab.findall('Colleague')
                    if colleagues:
                        strong_collabs = [c for c in colleagues if float(c.get('collaboration_strength', 0)) > 0.6]
                        print(f"  협업 동료: {len(colleagues)}명 (강한 협업: {len(strong_collabs)}명)")
                
                # 프로젝트 참여
                projects = emp_elem.find('ProjectRelationships')
                if projects is not None:
                    participations = projects.findall('ProjectParticipation')
                    active_projects = [p for p in participations if p.get('project_status') == 'active']
                    print(f"  참여 프로젝트: {len(participations)}개 (활성: {len(active_projects)}개)")
        
        print(f"\n=== XML 파일 검증 완료 ===")
        return True
        
    except Exception as e:
        print(f"XML 파일 검증 실패: {str(e)}")
        return False

# XML 검증 실행
validation_success = validate_and_preview_xml(XML_OUTPUT_PATH, num_employees_to_show=5)

if validation_success:
    print(f"\n✅ XML 관계 데이터 파일이 성공적으로 생성되었습니다!")
    print(f"   파일 위치: {XML_OUTPUT_PATH}")
    print(f"   JSON 버전: {JSON_OUTPUT_PATH}")
else:
    print(f"\n❌ XML 파일 생성에 문제가 있습니다.")

=== XML 파일 검증 결과 ===
루트 엘리먼트: OrganizationNetwork
생성 일시: 2025-09-01 20:46:30
총 직원 수: 1470
등록된 프로젝트: 25개
XML에 포함된 직원: 1470명

=== 샘플 직원 관계 정보 (상위 5명) ===

[직원 1: Employee_1]
  부서: Sales, 직무: Sales Executive, 위험도: High
  협업 동료: 443명 (강한 협업: 430명)
  참여 프로젝트: 0개 (활성: 0개)

[직원 2: Employee_2]
  부서: Research & Development, 직무: Research Scientist, 위험도: Stable
  협업 동료: 958명 (강한 협업: 906명)
  참여 프로젝트: 0개 (활성: 0개)

[직원 3: Employee_4]
  부서: Research & Development, 직무: Laboratory Technician, 위험도: High
  협업 동료: 930명 (강한 협업: 822명)
  참여 프로젝트: 0개 (활성: 0개)

[직원 4: Employee_5]
  부서: Research & Development, 직무: Research Scientist, 위험도: High
  협업 동료: 937명 (강한 협업: 820명)
  참여 프로젝트: 0개 (활성: 0개)

[직원 5: Employee_7]
  부서: Research & Development, 직무: Laboratory Technician, 위험도: Neutral
  협업 동료: 932명 (강한 협업: 837명)
  참여 프로젝트: 0개 (활성: 0개)

=== XML 파일 검증 완료 ===

✅ XML 관계 데이터 파일이 성공적으로 생성되었습니다!
   파일 위치: data\employee_relationships.xml
   JSON 버전: data\employee_relationships.json


특정 직원의 관계 조회 함수

In [7]:
# ------------------------------------------------------
# 특정 직원의 관계 조회 함수 (XML 활용 예시)
# ------------------------------------------------------

def query_employee_relationships(xml_file_path, employee_id):
    """특정 직원의 모든 관계 정보 조회"""
    
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        # 해당 직원 찾기
        employees_section = root.find('Employees')
        target_employee = None
        
        for emp in employees_section.findall('Employee'):
            if emp.get('id') == str(employee_id):
                target_employee = emp
                break
        
        if target_employee is None:
            print(f"직원 ID {employee_id}를 찾을 수 없습니다.")
            return None
        
        # 직원 기본 정보
        print(f"=== 직원 {employee_id} 관계 정보 ===")
        print(f"이름: {target_employee.get('name')}")
        print(f"부서: {target_employee.get('department')}")
        print(f"직무: {target_employee.get('job_role')}")
        print(f"직급: Level {target_employee.get('job_level')}")
        print(f"근속: {target_employee.get('years_at_company')}년")
        print(f"위험도: {target_employee.get('risk_tier')}")
        print(f"페르소나: {target_employee.get('persona')}")
        
        # 계층 관계
        hierarchy = target_employee.find('HierarchyRelationships')
        if hierarchy is not None:
            print(f"\n📋 계층 관계:")
            
            reports_to = hierarchy.find('ReportsTo')
            if reports_to is not None:
                print(f"  상사: {reports_to.get('manager_id')} ({reports_to.get('manager_role')})")
                print(f"       보고 빈도: {reports_to.get('reporting_frequency')}")
            else:
                print(f"  상사: 없음 (최고 관리층)")
            
            manages = hierarchy.find('Manages')
            if manages is not None:
                subordinates = manages.findall('Subordinate')
                print(f"  부하직원: {len(subordinates)}명")
                for sub in subordinates[:3]:  # 상위 3명만 표시
                    print(f"    - {sub.get('employee_id')} ({sub.get('job_role')})")
        
        # 협업 관계
        collab = target_employee.find('CollaborationRelationships')
        if collab is not None:
            colleagues = collab.findall('Colleague')
            print(f"\n🤝 협업 관계: {len(colleagues)}명")
            
            # 강한 협업 관계만 표시
            strong_collabs = [c for c in colleagues if float(c.get('collaboration_strength')) > 0.6]
            if strong_collabs:
                print(f"  주요 협업 동료 ({len(strong_collabs)}명):")
                for colleague in strong_collabs[:5]:  # 상위 5명
                    strength = colleague.get('collaboration_strength')
                    quality = colleague.get('relationship_quality')
                    print(f"    - {colleague.get('colleague_id')} ({colleague.get('colleague_role')}) "
                          f"- 강도: {strength}, 품질: {quality}")
        
        # 프로젝트 참여
        projects = target_employee.find('ProjectRelationships')
        if projects is not None:
            participations = projects.findall('ProjectParticipation')
            active_projects = [p for p in participations if p.get('project_status') == 'active']
            
            print(f"\n🚀 프로젝트 참여: {len(participations)}개")
            if active_projects:
                print(f"  현재 활성 프로젝트 ({len(active_projects)}개):")
                for project in active_projects:
                    proj_id = project.get('project_id')
                    role = project.get('role_in_project')
                    contribution = project.get('contribution_level')
                    print(f"    - {proj_id} (역할: {role}, 기여도: {contribution})")
                    
                    # 팀동료 정보
                    teammates_elem = project.find('Teammates')
                    if teammates_elem is not None:
                        teammates = teammates_elem.findall('Teammate')
                        teammate_names = [t.get('teammate_id') for t in teammates]
                        print(f"      팀동료: {', '.join(teammate_names)}")
        
        return target_employee
        
    except Exception as e:
        print(f"관계 조회 실패: {str(e)}")
        return None

# 사용 예시: 랜덤한 직원 3명의 관계 조회
sample_employee_ids = df['EmployeeNumber'].sample(3).tolist()

for emp_id in sample_employee_ids:
    query_employee_relationships(XML_OUTPUT_PATH, emp_id)
    print("\n" + "="*60 + "\n")

=== 직원 1824 관계 정보 ===
이름: Employee_1824
부서: Sales
직무: Manager
직급: Level 4
근속: 16년
위험도: High
페르소나: Career Stall (성장 정체)

📋 계층 관계:
  상사: 1591 (Manager)
       보고 빈도: weekly
  부하직원: 4명
    - 864 (Sales Executive)
    - 1761 (Sales Executive)
    - 207 (Sales Executive)

🤝 협업 관계: 443명
  주요 협업 동료 (351명):
    - 1 (Sales Executive) - 강도: 0.7, 품질: good
    - 23 (Manager) - 강도: 1.0, 품질: good
    - 35 (Sales Executive) - 강도: 0.818, 품질: excellent
    - 38 (Manager) - 강도: 1.0, 품질: fair
    - 42 (Sales Representative) - 강도: 0.665, 품질: fair

🚀 프로젝트 참여: 0개


=== 직원 72 관계 정보 ===
이름: Employee_72
부서: Sales
직무: Sales Executive
직급: Level 2
근속: 2년
위험도: High
페르소나: Burnout (저성과자/번아웃)

📋 계층 관계:
  상사: 없음 (최고 관리층)

🤝 협업 관계: 444명
  주요 협업 동료 (430명):
    - 1 (Sales Executive) - 강도: 1.0, 품질: good
    - 27 (Sales Representative) - 강도: 0.68, 품질: fair
    - 35 (Sales Executive) - 강도: 1.0, 품질: excellent
    - 42 (Sales Representative) - 강도: 0.85, 품질: fair
    - 47 (Sales Representative) - 강도: 1.0, 품질: excellent

🚀 프로젝트